### Get Mapping data to get player ID

In [8]:
import json
import pandas as pd

# Load the JSON data for the game events and the mapping file
file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/val:f567361b-8d9d-4f3e-84f6-090ae8499b45.json"
mapping_file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/mapping_data.json"

with open(file_path, 'r') as f:
    data = json.load(f)

with open(mapping_file_path, 'r') as mf:
    mapping_data = json.load(mf)

# Initialize dictionaries to store kill, death, and assist counts
kill_counts = {}
death_counts = {}
assist_counts = {}

# Helper function to map player ID using the platformGameId and participantMapping
def map_player_id(platform_game_id, player_id, mappings):
    # Find the mapping for the platformGameId
    for mapping in mappings:
        if mapping['platformGameId'] == platform_game_id:
            # Return the corresponding esports player ID from participantMapping
            return mapping['participantMapping'].get(str(player_id), player_id)  # Return player_id if not found
    return player_id  # Return the original player_id if platformGameId is not found

# Iterate through the events and count kills, deaths, and assists
for event in data:
    platform_game_id = event['platformGameId']
    
    if 'playerDied' in event:
        deceased_id = event['playerDied']['deceasedId']['value']
        killer_id = event['playerDied']['killerId']['value']
        
        # Map deceased and killer IDs using the participantMapping
        mapped_deceased_id = map_player_id(platform_game_id, deceased_id, mapping_data)
        mapped_killer_id = map_player_id(platform_game_id, killer_id, mapping_data)

        # Increment death count for deceased player
        if mapped_deceased_id in death_counts:
            death_counts[mapped_deceased_id] += 1
        else:
            death_counts[mapped_deceased_id] = 0  # start at 1, not 0

        # Increment kill count for killer
        if mapped_killer_id in kill_counts:
            kill_counts[mapped_killer_id] += 1
        else:
            kill_counts[mapped_killer_id] = 0  # start at 1, not 0

        # Check for assistants and update their assist counts
        if 'assistants' in event['playerDied']:
            for assistant in event['playerDied']['assistants']:
                assistant_id = assistant['assistantId']['value']
                mapped_assistant_id = map_player_id(platform_game_id, assistant_id, mapping_data)
                
                if mapped_assistant_id in assist_counts:
                    assist_counts[mapped_assistant_id] += 1
                else:
                    assist_counts[mapped_assistant_id] = 0

# Combine the results into a dataframe for better readability
kill_death_df = pd.DataFrame(list(kill_counts.keys()), columns=["PlayerId"])
kill_death_df["Kills"] = kill_death_df["PlayerId"].map(kill_counts)
kill_death_df["Deaths"] = kill_death_df["PlayerId"].map(death_counts)
kill_death_df["Assists"] = kill_death_df["PlayerId"].map(assist_counts).fillna(0).astype(int)  # fill missing assists with 0

# Output the DataFrame
print(kill_death_df)


             PlayerId  Kills  Deaths  Assists
0  106977390661334163      7       9        6
1  107186009533585182     10      15        1
2  106977394394708775     15      10        4
3  106977393332501282     14       5        2
4  106977392050144626     15       8        6
5  107186009230387255     12      13        1
6  106977457782256833     17       5        9
7  107025879641342377      4      14        6
8  109080898754120338      9      13        4
9  107186009028237077      2      13        6


### Get player with latest update time

In [9]:
import pandas as pd
import json

# Load player data
player_file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/players.json"
with open(player_file_path, 'r') as pf:
    player_data = json.load(pf)

# Normalize and clean player data
player_df = pd.DataFrame(player_data)

# Remove leading/trailing spaces and lowercase the names for consistency
player_df['first_name'] = player_df['first_name'].str.strip().str.title()
player_df['last_name'] = player_df['last_name'].str.strip().str.title()
player_df['handle'] = player_df['handle'].str.strip().str.title()

# Convert 'updated_at' toi datetime for sorting
player_df['updated_at'] = pd.to_datetime(player_df['updated_at'])
# Sort by 'updated_at' in descending order so the most recent entry comes first
player_df = player_df.sort_values(by='updated_at', ascending=False)

# Group by 'id' and select the first entry (which will be the most recent due to sorting)
player_df = player_df.groupby('id').first().reset_index()

# Select relevant columns
player_df = player_df[['id', 'handle', 'first_name', 'last_name','home_team_id']]
print(player_df)
# Match the player information with the kill_death_df using 'PlayerId'
# Assuming kill_death_df is the DataFrame from the earlier step containing player stats
kda_df = kill_death_df.merge(player_df, left_on='PlayerId', right_on='id', how='left')

# Drop the 'id' column as it is redundant after the merge
kda_df = kda_df.drop(columns=['id'])

# Display the final DataFrame with handle, first_name, last_name, and the stats
print(kda_df)

                      id     handle first_name  last_name        home_team_id
0     106172873671772761     Sickly       Juan       Juan  107910153514729818
1     106173004258282083      Fr1Xx    Rodrigo      Vivas  107605767634609340
2     106229939974586995     Vanity    Anthony  Malaspina  106013963939603177
3     106229941209880276      Effys       Loic  Sauvageau  106013963939603177
4     106229947030626124      Penny       Erik      Penny  106013963939603177
...                  ...        ...        ...        ...                 ...
1507  113038806122799029      Noor1       Noor   Almegren  110196054985503335
1508  113038949666013843    Susbake      Kayla    Hartley  110614998843401533
1509  113038968137007776     Dream1     Louise         Xu  111890685909663613
1510  113068485580341633  Nescargot       Inès     Gorlez  111006791685855925
1511  113068487057105034        Bee        Bee   Bradbury  112491034388390805

[1512 rows x 5 columns]
             PlayerId  Kills  Deaths  A

Connect team and leagues

In [10]:
import json

# Load the team data
with open('/Users/leon/Documents/GitHub/VCTEVA/Test/teams.json', 'r') as team_file:
    teams = json.load(team_file)

# Load the league data
with open('/Users/leon/Documents/GitHub/VCTEVA/Test/leagues.json', 'r') as league_file:
    leagues = json.load(league_file)

# Create a dictionary to map league_id to league details
league_dict = {league['league_id']: league for league in leagues}

# List to hold the new matched data
matched_data = []

# Match teams with their corresponding leagues
for team in teams:
    home_league_id = team['home_league_id']
    if home_league_id in league_dict:
        league = league_dict[home_league_id]
        matched_data.append({
            "team_id": team['id'],
            "acronym": team['acronym'],
            "league_id": league['league_id'],
            "team_name": team['name'],
            "league_region": league['region'],
            "league_name": league['name']
        })

# Write the matched data to a new JSON file
with open('matched_teams_leagues.json', 'w') as output_file:
    json.dump(matched_data, output_file, indent=4)

print("New JSON file with matched data created.")


New JSON file with matched data created.


In [11]:
import pandas as pd
matched_data_df = pd.DataFrame(matched_data)
print(matched_data_df)

                team_id acronym           league_id          team_name  \
0    111006711389612074      LH  109029777807406730   Lunatic-hai flax   
1    109029888698112406     LWE  109029777807406730  Lone Way E-SPORTS   
2    112439820691490172     MIR  109029777807406730         MIR Gaming   
3    112439825505751243     STG  109029777807406730  Special TiGers GC   
4    112439841344832900     PIX  109029777807406730              Pixel   
..                  ...     ...                 ...                ...   
462  107605767634609340     KRÜ  109879575186437267     VISA KRÜ Blaze   
463  112127238709173458     MVR  109879575186437267              MYVRA   
464  112127272470502649     KR7  109879575186437267            Killer7   
465  107605767634609340     KRU  109879575186437267        KRÜ Esports   
466  108452406268738690     9ZG  109879575186437267         9z Globant   

    league_region          league_name  
0              KR     game_changers_kr  
1              KR     game_ch

Connect player, team, region

In [14]:

final_df = kda_df.merge(matched_data_df, left_on='home_team_id', right_on='team_id', how='left')
# 删除不需要的列 'home_team_id'
final_df = final_df.drop(columns=['home_team_id']) 


final_df

,PlayerId,Kills,Deaths,Assists,handle,first_name,last_name,team_id,acronym,league_id,team_name,league_region,league_name
0,106977390661334163,7,9,6,Katsumi,Kat,,108065567071390915,XSET,106976737954740691,XSET Purple,NA,game_changers_na
1,106977390661334163,7,9,6,Katsumi,Kat,,108065567071390915,XSET,106976737954740691,XSET,NA,game_changers_na
2,106977390661334163,7,9,6,Katsumi,Kat,,108065567071390915,XSET,106976737954740691,XSET Fe,NA,game_changers_na
3,107186009533585182,10,15,1,Smurfette,Yağmur,Gündüz,107021298845350518,FUT,107019646737643925,Futbolist JEFF,EMEA,game_changers_emea
4,107186009533585182,10,15,1,Smurfette,Yağmur,Gündüz,107021298845350518,FUT,107019646737643925,FUT Female,EMEA,game_changers_emea
5,107186009533585182,10,15,1,Smurfette,Yağmur,Gündüz,107021298845350518,FUT,107019646737643925,Futbolist Female,EMEA,game_changers_emea
6,106977394394708775,15,10,4,Jazzyk1Ns,Jasmine,Manankil,112002552616520453,PP,106976737954740691,Passion Project,NA,game_changers_na
7,106977393332501282,14,5,2,Mel,Melanie,Capone,106013963939603177,V1,106976737954740691,Version 1,NA,game_changers_na
8,106977392050144626,15,8,6,Alexis,Alexis,Guarrasi,106976742467025364,SR,106976737954740691,Shopify Rebellion,NA,game_changers_na
9,106977392050144626,15,8,6,Alexis,Alexis,Guarrasi,106976742467025364,SR,106976737954740691,Shopify Rebellion,NA,game_changers_na


TODO: 怎么根据类似以下数据给kda按照攻守和地图分类

"configuration": {"teams": [{"teamId": {"value": 21}, "playersInTeam": [{"value": 6}, {"value": 7}, {"value": 8}, {"value": 9}, {"value": 10}], "name": "Red"}, {"teamId": {"value": 22}, "playersInTeam": [{"value": 1}, {"value": 2}, {"value": 3}, {"value": 4}, {"value": 5}], "name": "Blue"}], "selectedMap": {"id": "UNKNOWN", "fallback": {"guid": "/Game/Maps/Canyon/Canyon", "displayName": "Canyon"}